In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tomopy
import tifffile
from skimage.metrics import structural_similarity as ssim

In [8]:
data1 = tifffile.imread('/data/xrf_tomo_andy/exp_202211_sim/exp4_Ni_K_ganrec-1.tif')
data2 = tifffile.imread('/data/xrf_tomo_andy/exp_202211_sim/exp4_Ni_K_ganrec-2.tif')

In [9]:
data3 = tifffile.imread('/data/xrf_tomo_andy/exp_202211_sim/data-1.tif')
data4 = tifffile.imread('/data/xrf_tomo_andy/exp_202211_sim/data.tiff')

In [10]:
data = data3+data4
data[data<0.1]=0
tifffile.imsave('/data/xrf_tomo_andy/exp_202211_sim/data_1.tiff', data)
ang = tomopy.angles(181)
prj = tomopy.project(data, ang, pad=False)
tifffile.imsave('/data/xrf_tomo_andy/exp_202211_sim/prj_sim.tiff', prj)

In [12]:
def nor_data(data):
    data = (data-data.min())/(data.max()-data.min())
    return data

In [8]:
prj = nor_data(prj)

In [14]:
def mssim_compute(data1, data2):
    mssim = np.zeros((len(data)))
    data1 = nor_data(data1)
    data2 = nor_data(data2)
    for i in range(len(data1)):
        ssim_val = ssim(data1[i], data2[i], full = True)
        mssim[i] = ssim_val[0]
    return mssim
    

In [15]:

for i in range(10):
    ang = tomopy.angles(18*(i+1)+1)
    prj = tomopy.project(data, ang, pad=False)
    prj = nor_data(prj)
    # tifffile.imsave(f'/data/xrf_tomo_andy/exp_202211_sim/prj_sim_{18*(i+1)+1}.tiff', prj)
    recon_gridrec = tomopy.recon(prj, ang, algorithm='gridrec', filter_name='parzen')
    tifffile.imsave(f'/data/xrf_tomo_andy/exp_202211_sim/recon/recon_gridrec_{18*(i+1)+1}.tiff', recon_gridrec)
    mssim_gridrec = mssim_compute(data, recon_gridrec)
    recon_sirt = tomopy.recon(prj, ang, algorithm='sirt', num_iter =100)
    tifffile.imsave(f'/data/xrf_tomo_andy/exp_202211_sim/recon/recon_sirt_{18*(i+1)+1}.tiff', recon_sirt)
    mssim_sirt = mssim_compute(data, recon_sirt)
    recon_mlem = tomopy.recon(prj, ang, algorithm='pml_hybrid', num_iter =100)
    tifffile.imsave(f'/data/xrf_tomo_andy/exp_202211_sim/recon/recon_pml_hybrid_{18*(i+1)+1}.tiff', recon_mlem)
    mssim_pml = mssim_compute(data, recon_mlem)

In [20]:
recon_pml = tifffile.imread('/data/xrf_tomo_andy/exp_202211_sim/recon/recon_pml_hybrid_19.tiff')

In [21]:
mssim_pml_19 = mssim_compute(data, recon_pml)

In [24]:
print(len(ang))

181


In [14]:
for i in range(18):
    ang = np.concatenate((tomopy.angles(i+1, ang1=0, ang2=10*(i+1)), tomopy.angles(18-i, ang1=10*(i+2), ang2=180)))
    prj = tomopy.project(data, ang, pad=False)
    prj = nor_data(prj)
    # # tifffile.imsave(f'/data/xrf_tomo_andy/exp_202211_sim/prj_sim_{18*(i+1)+1}.tiff', prj)
    # recon_gridrec = tomopy.recon(prj, ang, algorithm='gridrec', filter_name='parzen')
    # tifffile.imsave(f'/data/xrf_tomo_andy/exp_202211_sim/recon/recon_gridrec_{18*(i+1)+1}.tiff', recon_gridrec)
    # mssim_gridrec = mssim_compute(data, recon_gridrec)
    # recon_sirt = tomopy.recon(prj, ang, algorithm='sirt', num_iter =100)
    # tifffile.imsave(f'/data/xrf_tomo_andy/exp_202211_sim/recon/recon_sirt_{18*(i+1)+1}.tiff', recon_sirt)
    # mssim_sirt = mssim_compute(data, recon_sirt)
    recon_mlem = tomopy.recon(prj, ang, algorithm='pml_hybrid', num_iter =100)
    tifffile.imsave(f'/data/xrf_tomo_andy/exp_202211_sim/recon/nonuniform/recon_pml_hybrid_{10*(i+1)+1}.tiff', recon_mlem)
    # mssim_pml = mssim_compute(data, recon_mlem)
    print(ang)

[0.         0.34906585 0.51333213 0.67759842 0.8418647  1.00613098
 1.17039726 1.33466355 1.49892983 1.66319611 1.82746239 1.99172868
 2.15599496 2.32026124 2.48452752 2.64879381 2.81306009 2.97732637
 3.14159265]
[0.         0.34906585 0.52359878 0.68722339 0.85084801 1.01447263
 1.17809725 1.34172186 1.50534648 1.6689711  1.83259571 1.99622033
 2.15984495 2.32346957 2.48709418 2.6507188  2.81434342 2.97796804
 3.14159265]
[0.         0.26179939 0.52359878 0.6981317  0.8610291  1.02392649
 1.18682389 1.34972129 1.51261869 1.67551608 1.83841348 2.00131088
 2.16420827 2.32710567 2.49000307 2.65290046 2.81579786 2.97869526
 3.14159265]
[0.         0.23271057 0.46542113 0.6981317  0.87266463 1.03473091
 1.1967972  1.35886349 1.52092978 1.68299606 1.84506235 2.00712864
 2.16919493 2.33126122 2.4933275  2.65539379 2.81746008 2.97952637
 3.14159265]
[0.         0.21816616 0.43633231 0.65449847 0.87266463 1.04719755
 1.20830487 1.36941218 1.5305195  1.69162681 1.85273413 2.01384144
 2.1749487

In [4]:
ang = tomopy.angles(19)
ang

array([0.        , 0.17453293, 0.34906585, 0.52359878, 0.6981317 ,
       0.87266463, 1.04719755, 1.22173048, 1.3962634 , 1.57079633,
       1.74532925, 1.91986218, 2.0943951 , 2.26892803, 2.44346095,
       2.61799388, 2.7925268 , 2.96705973, 3.14159265])